# Movielens 영화 추천

## 1) 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [69]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


## 2) MF model 구성하기

In [8]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [9]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

## 3) 분석 하기

### 3-1. ratings에 있는 유니크한 영화 개수

In [10]:
ratings['movie_id'].nunique()

3628

### 3-2. ratings에 있는 유니크한 사용자 수

In [11]:
ratings['user_id'].nunique()

6039

### 3-3. 가장 인기 있는 영화 30개(인기순)

In [12]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [13]:
movie_id_list = list(ratings['movie_id'])

print(movies['title'][movies['movie_id'] == 1721].values[0])


print(movies['movie_id'][movies['title'] == 'Titanic (1997)'].values[0])



Titanic (1997)
1721


In [14]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_top_30_list = list(movie_count.sort_values(ascending=False).head(30).index)


for id in movie_top_30_list:
    print(movies['title'][movies['movie_id'] == id].values[0])

American Beauty (1999)
Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode VI - Return of the Jedi (1983)
Saving Private Ryan (1998)
Terminator 2: Judgment Day (1991)
Silence of the Lambs, The (1991)
Raiders of the Lost Ark (1981)
Back to the Future (1985)
Matrix, The (1999)
Jurassic Park (1993)
Sixth Sense, The (1999)
Fargo (1996)
Braveheart (1995)
Men in Black (1997)
Schindler's List (1993)
Princess Bride, The (1987)
Shakespeare in Love (1998)
L.A. Confidential (1997)
Shawshank Redemption, The (1994)
Godfather, The (1972)
Groundhog Day (1993)
E.T. the Extra-Terrestrial (1982)
Being John Malkovich (1999)
Ghostbusters (1984)
Pulp Fiction (1994)
Forrest Gump (1994)
Terminator, The (1984)
Toy Story (1995)
Fugitive, The (1993)


## 4) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [75]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [73]:
new_ratings = pd.merge(left = ratings,right = movies, on='movie_id')

In [74]:
new_ratings

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


ratings의 맨 마지막 id보다 1을 더한값으로 나의 id를 생성한다.

In [76]:
my_id = '재영'

my_favorite = ['Titanic (1997)' , 'Toy Story (1995)' ,'Lion King, The (1994)' ,'Superman (1978)' ,'Scream 2 (1997)']

my_timestamp = [i + 211026429 for i in range(5)]

In [77]:
my_playlist = pd.DataFrame({'user_id': [my_id]*5, 'movie_id': my_favorite, 'count':[5,3,5,4,3], 'timestamp': my_timestamp})


In [78]:
my_playlist

,user_id,movie_id,count,timestamp
0,재영,Titanic (1997),5,211026429
1,재영,Toy Story (1995),3,211026430
2,재영,"Lion King, The (1994)",5,211026431
3,재영,Superman (1978),4,211026432
4,재영,Scream 2 (1997),3,211026433


In [79]:
ratings = ratings.append(my_playlist) 

In [80]:
ratings.tail(10)

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569
0,재영,Titanic (1997),5,211026429
1,재영,Toy Story (1995),3,211026430
2,재영,"Lion King, The (1994)",5,211026431
3,재영,Superman (1978),4,211026432
4,재영,Scream 2 (1997),3,211026433


## 5) CSR matrix를 직접 만들어 봅시다.

In [81]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [82]:

temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')


temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5,978300760
1,0,1,3,978302109
2,0,2,3,978301968
3,0,3,4,978300275
4,0,4,5,978824291
...,...,...,...,...
0,6039,3628,5,211026429
1,6039,3629,3,211026430
2,6039,3630,5,211026431
3,6039,3631,4,211026432


In [83]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 6) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [84]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [85]:
print(csr_data)
csr_data_transpose = csr_data.T
csr_data_transpose

  (0, 0)	5
  (0, 1)	3
  (0, 2)	3
  (0, 3)	4
  (0, 4)	5
  (0, 5)	3
  (0, 6)	5
  (0, 7)	5
  (0, 8)	4
  (0, 9)	4
  (0, 10)	5
  (0, 11)	4
  (0, 12)	4
  (0, 13)	4
  (0, 14)	5
  (0, 15)	4
  (0, 16)	3
  (0, 17)	4
  (0, 18)	5
  (0, 19)	4
  (0, 20)	3
  (0, 21)	3
  (0, 22)	5
  (0, 23)	5
  (0, 24)	3
  :	:
  (6038, 2311)	4
  (6038, 2317)	5
  (6038, 2386)	4
  (6038, 2394)	5
  (6038, 2424)	4
  (6038, 2437)	4
  (6038, 2446)	5
  (6038, 2471)	4
  (6038, 2511)	5
  (6038, 2523)	4
  (6038, 2559)	3
  (6038, 2560)	4
  (6038, 2631)	5
  (6038, 2648)	4
  (6038, 2654)	5
  (6038, 2738)	4
  (6038, 2740)	5
  (6038, 2857)	5
  (6038, 2860)	3
  (6038, 3311)	5
  (6039, 3628)	5
  (6039, 3629)	3
  (6039, 3630)	5
  (6039, 3631)	4
  (6039, 3632)	3


<3633x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [86]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [87]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [94]:
jy, titanic = user_to_idx['재영'], movie_to_idx['Superman (1978)']
jy_vector, titanic_vector = als_model.user_factors[jy], als_model.item_factors[titanic]

In [95]:
jy_vector

array([ 0.0150288 , -0.01618455,  0.01035406, -0.004964  , -0.01242414,
       -0.00630189, -0.00113261,  0.00458865, -0.00716187, -0.00966665,
        0.00746161,  0.01953897,  0.00690887,  0.0016485 ,  0.00823795,
        0.01801115,  0.00341546,  0.01129826, -0.00045896, -0.0033697 ,
        0.00269226,  0.00985128,  0.01006526,  0.01878523, -0.00842059,
       -0.00909623,  0.01085127, -0.00544911,  0.00930824,  0.0063379 ,
       -0.01213942,  0.0004164 , -0.00802269, -0.00644377, -0.00494617,
       -0.01235625, -0.00049521, -0.00031937, -0.00185734, -0.00249199,
       -0.0031663 , -0.02075731, -0.00014422,  0.01170541, -0.01893477,
        0.00312248, -0.00466007,  0.03154242, -0.00229066,  0.01102897,
       -0.02440351,  0.00339506, -0.00108111,  0.01070657, -0.01642878,
        0.00394325, -0.00332699,  0.00461324, -0.00025049,  0.00913747,
        0.00576221, -0.00897655, -0.01048551,  0.00881991, -0.00648052,
       -0.00142468,  0.0020387 , -0.00885731,  0.00253917, -0.01

In [96]:
titanic_vector

array([ 6.50706759e-04, -1.23976031e-04,  2.80269189e-04,  2.69633048e-04,
        1.69526262e-04,  1.09712921e-04,  3.33958887e-04,  2.28164296e-04,
        2.85015199e-06, -3.63946383e-05,  2.22048897e-04,  5.36079344e-04,
        2.78316264e-04,  2.79671804e-04,  4.34241811e-04,  5.70747361e-04,
        2.77062558e-04,  3.43374122e-04,  3.53444542e-04,  3.20889521e-04,
        3.79182980e-04,  4.67192207e-04,  4.20332770e-04,  7.21327029e-04,
        3.04394809e-04, -5.65178598e-05,  4.58669150e-04,  1.64604178e-04,
        3.24599794e-04,  3.36505997e-04, -1.34086522e-05,  2.14516884e-04,
        9.70686815e-05,  3.90334426e-05,  1.66181373e-04,  8.85836780e-05,
        3.36726633e-04,  2.06945100e-04,  1.03171427e-04,  6.27760164e-05,
        2.88271141e-04, -1.07719687e-04,  2.05548451e-04,  5.52142854e-04,
       -1.22435187e-04,  3.47594760e-04,  2.04168682e-04,  8.97252758e-04,
        2.43070783e-04,  4.12394758e-04, -2.29004698e-04,  1.38739662e-04,
        2.36730528e-04,  

In [98]:
np.dot(jy_vector, titanic_vector)

0.00018319815

## 7) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

## 8) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [102]:
favorite_movie = 'Lion King, The (1994)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3630, 1.0),
 (3629, 0.9999335),
 (3628, 0.9991766),
 (3632, 0.99473315),
 (3631, 0.99111676),
 (3576, 0.97241217),
 (3575, 0.9706465),
 (3574, 0.969008),
 (3578, 0.9688826),
 (3580, 0.96827567),
 (3572, 0.96785986),
 (3577, 0.9667995),
 (3583, 0.96672606),
 (3579, 0.96618766),
 (3573, 0.96568024)]

In [103]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Lion King, The (1994)',
 'Toy Story (1995)',
 'Titanic (1997)',
 'Scream 2 (1997)',
 'Superman (1978)',
 1548,
 1510,
 607,
 2214,
 3229,
 584,
 701,
 138,
 763,
 2821]

In [105]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [107]:
get_similar_movie('Superman (1978)')

['Superman (1978)',
 'Scream 2 (1997)',
 'Toy Story (1995)',
 'Lion King, The (1994)',
 'Titanic (1997)',
 1510,
 1548,
 2214,
 3229,
 607]